In [1]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc
import helper

### Exercise 1

Find the uncompressed SEC format for the Public Key where the Private Key secrets are:

* 5000
* \\(2018^{5}\\)
* 0xdeadbeef12345

In [17]:
# Exercise 1

from ecc import PrivateKey

problems = [ 5000, 2018**5, 0xdeadbeef12345 ]

for secret in problems:
    P = PrivateKey(secret).point
    usec = b'\x04' + P.x.num.to_bytes(32, 'big') + P.y.num.to_bytes(32, 'big')
    print(usec.hex())


04ffe558e388852f0120e46af2d1b370f85854a8eb0841811ece0e3e03d282d57c315dc72890a4f10a1481c031b03b351b0dc79901ca18a00cf009dbdb157a1d10
04027f3da1918455e03c46f659266a1bb5204e959db7364d2f473bdf8f0a13cc9dff87647fd023c13b4a4994f17691895806e1b40b57f4fd22581a4f46851f3b06
04d90cd625ee87dd38656dd95cf79f65f60f7273b67d3096e68bd81e4f5342691f842efa762fd59961d0e99803c61edba8b3e3f7dc3a341836f97733aebf987121


### Exercise 2

Find the Compressed SEC format for the Public Key where the Private Key secrets are:

* 5001
* \\(2019^{5}\\)
* 0xdeadbeef54321

In [41]:
# Exercise 2

from ecc import PrivateKey, S256Field, S256Point

B = 7
Prime = 2**256 - 2**32 - 977

problems = [ 5001, 2019**5, 0xdeadbeef54321 ]

def parse(bin):
    is_even = bin[0] == 2
    x = S256Field(int.from_bytes(bin[1:], 'big'))
    alpha = x**3 + S256Field(7)
    beta = alpha ** ((Prime + 1) // 4)
    if beta.num % 2 == 0:
        even_beta = beta
        odd_beta  = S256Field(Prime - beta.num)
    else:
        even_beta = S256Field(Prime - beta.num)
        odd_beta  = beta
    if is_even:
        return S256Point(x, even_beta)
    else:
        return S256Point(x, odd_beta)

for secret in problems:
    P = PrivateKey(secret).point
    print("Before compression: {}\n".format(P))
    if (P.y.num % 2 == 0):
        sec = b'\x02' + P.x.num.to_bytes(32, 'big')
    else:
        sec = b'\x03' + P.x.num.to_bytes(32, 'big')
    print("Compressed: {}\n".format(sec.hex()))
    
    print("After compression: {}\n".format(parse(sec)))

Before compression: S256Point(57a4f368868a8a6d572991e484e664810ff14c05c0fa023275251151fe0e53d1, 0d6cc87c5bc29b83368e17869e964f2f53d52ea3aa3e5a9efa1fa578123a0c6d)

Compressed: 0357a4f368868a8a6d572991e484e664810ff14c05c0fa023275251151fe0e53d1

After compression: S256Point(57a4f368868a8a6d572991e484e664810ff14c05c0fa023275251151fe0e53d1, 0d6cc87c5bc29b83368e17869e964f2f53d52ea3aa3e5a9efa1fa578123a0c6d)

Before compression: S256Point(933ec2d2b111b92737ec12f1c5d20f3233a0ad21cd8b36d0bca7a0cfa5cb8701, 96cbbfdd572f75ace44d0aa59fbab6326cb9f909385dcd066ea27affef5a488c)

Compressed: 02933ec2d2b111b92737ec12f1c5d20f3233a0ad21cd8b36d0bca7a0cfa5cb8701

After compression: S256Point(933ec2d2b111b92737ec12f1c5d20f3233a0ad21cd8b36d0bca7a0cfa5cb8701, 96cbbfdd572f75ace44d0aa59fbab6326cb9f909385dcd066ea27affef5a488c)

Before compression: S256Point(96be5b1292f6c856b3c5654e886fc13511462059089cdf9c479623bfcbe77690, 32555d1b027c25c2828ba96a176d78419cd1236f71558f6187aec09611325eb6)

Compressed: 0296be5b1292f6c

### Exercise 3

Find the DER format for a signature whose `r` and `s` values are:

* r =

`0x37206a0610995c58074999cb9767b87af4c4978db68c06e8e6e81d282047a7c6`

* s =

`0x8ca63759c1157ebeaec0d03cecca119fc9a75bf8e6d0fa65c841c8e2738cdaec`

In [66]:
# Exercise 3

from ecc import Signature

r = 0x37206a0610995c58074999cb9767b87af4c4978db68c06e8e6e81d282047a7c6
s = 0x8ca63759c1157ebeaec0d03cecca119fc9a75bf8e6d0fa65c841c8e2738cdaec

def encode(num):
    nbin = num.to_bytes(32, 'big').lstrip(b'\x00')
    ## If nbin starts with a high-bit, add a '00' buffer.
    if nbin[0] & 0x80:
        nbin = b'\x00' + nbin
    return bytes([2, len(nbin)]) + nbin

formatted = encode(r) + encode(s)
full_der = bytes([0x30, len(formatted)]) + formatted

print(full_der.hex())




Length of encoding: 32
Length of encoding: 33
3045022037206a0610995c58074999cb9767b87af4c4978db68c06e8e6e81d282047a7c60221008ca63759c1157ebeaec0d03cecca119fc9a75bf8e6d0fa65c841c8e2738cdaec


### Exercise 4

Convert the following hex to binary and then to Base58:

* `7c076ff316692a3d7eb3c3bb0f8b1488cf72e1afcd929e29307032997a838a3d`
* `eff69ef2b1bd93a66ed5219add4fb51e11a840f404876325a1e8ffe0529a2c`
* `c7207fee197d27c618aea621406f6bf5ef6fca38681d82b2f06fddbdce6feab6`

In [75]:
# Exercise 4

## from helper import encode_base58

def encode_base58(s):
    ALPHABET = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'
    count = 0
    for c in s:
        if c == 0:
            count += 1
        else:
            break
    num = int.from_bytes(s, 'big')
    prefix = '1' * count
    result = ''
    while num > 0:
        num, mod = divmod(num, 58)
        result = ALPHABET[mod] + result
    return prefix + result

problems = [
    0x7c076ff316692a3d7eb3c3bb0f8b1488cf72e1afcd929e29307032997a838a3d,
    0xeff69ef2b1bd93a66ed5219add4fb51e11a840f404876325a1e8ffe0529a2c,
    0xc7207fee197d27c618aea621406f6bf5ef6fca38681d82b2f06fddbdce6feab6
]

for prob in problems:
    nbin = prob.to_bytes(32, 'big')
    print(encode_base58(nbin))

9MA8fRQrT4u8Zj8ZRd6MAiiyaxb2Y1CMpvVkHQu5hVM6
14fE3H2E6XMp4SsxtwinF7w9a34ooUrwWe4WsW1458Pd
EQJsjkd6JaGwxrjEhfeqPenqHwrBmPQZjJGNSCHBkcF7


### Exercise 5

Find the address corresponding to Public Keys whose Private Key secrets are:

* 5002 (use uncompressed SEC, on testnet)
* \\(2020^{5}\\) (use compressed SEC, on testnet)
* 0x12345deadbeef (use compressed SEC on mainnet)

In [125]:
# Exercise 5
import hashlib

from ecc       import PrivateKey
from helper    import encode_base58
from ripemd160 import ripemd160

def hash256(s):
    return hashlib.sha256(s).digest()

def hash160(s):
    return ripemd160(s)

problems = [ 
    [ 5002, False, 'testnet' ],
    [ 2020**5, True, 'testnet'],
    [ 0x12345deadbeef, True, 'mainnet' ]
]

for prob in problems:
    key, compressed, network = prob
    
    ## Generate keypair.
    e = PrivateKey(key)
    
    ## Get prefix code:
    if network == 'testnet':
        prefix = bytes([0x6f])
    else:
        prefix = bytes([0x00])
    
    ## Serialize the pubkey (point) using SEC encoding,
    ## then sha256 + ripemd160 hash the result.
    encoded = hash160(hash256(e.point.sec(compressed)))
    
    ## Add the prefix to the encoded bytes.
    binary  = prefix + encoded
    
    ## Hash the combined binary and get the first 4 bytes as a checksum.
    checksum = hash256(binary)[0:4]
    
    ## Combine binary and checksum, then encode in base58.
    address = encode_base58(binary + checksum)
    
    print(address)

mmTPbXQFxboEtNRkwfh6K51jvdtHKqy6Qh
mopVkxp8UhXqRYbCYJsbeE1h1fiF19dM2S
1F1Pn2y6pDb68E5nYJJeba4TLg2U9QHEgK


### Exercise 6

Find the WIF for Private Key whose secrets are:

* 5003 (compressed, testnet)
* \\(2021^{5}\\) (uncompressed, testnet)
* 0x54321deadbeef (compressed, mainnet)

In [126]:
# Exercise 6

from ecc import PrivateKey

# 5003
# 2021**5
# 0x54321deadbeef

problems = [ 
    [ 5003, True, 'testnet' ],
    [ 2021**5, False, 'testnet'],
    [ 0x54321deadbeef, True, 'mainnet' ]
]

for prob in problems:
    key, compressed, network = prob
    
    # Create the binary string from the key.
    secret_bytes = key.to_bytes(32, 'big')
    
    ## Get prefix code:
    if network == 'testnet':
        prefix = b'\xef'
    else:
        prefix = b'\x80'
        
    ## Add suffix code if compressed.
    if compressed:
        suffix = b'\x01'
    else:
        suffix = b''
    
    ## Concatenate all the bytes together.
    all_bytes = prefix + secret_bytes + suffix
    
    ## Hash the combined output and get the first 4 bytes as a checksum.
    checksum = hash256(all_bytes)[0:4]
    
    ## Encode the result in base58.
    wif_format = encode_base58(all_bytes + checksum)
    
    print(wif_format)
    
    
    

cMahea7zqjxrtgAbB7LSGbcQUr1uX1ojuat9jZodMN8rFTs6w9Ag
91avARGdfge8E4tZfYLoxeJ5sGBdNJQH4kvjpWAxgzczjZ74iEV
KwDiBf89QgGbjEhKnhXJuH7LrciVrZi3qYjgiuQJv1h8Ytpv4H7U


### Exercise 7

Write a function `little_endian_to_int` which takes Python bytes, interprets those bytes in Little-Endian and returns the number.

#### Make [this test](/edit/code-ch04/helper.py) pass: `helper.py:HelperTest:test_little_endian_to_int`

In [127]:
# Exercise 7

reload(helper)
run(helper.HelperTest("test_little_endian_to_int"))

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


### Exercise 8

Write a function `int_to_little_endian` which does the reverse of the last exercise.

#### Make [this test](/edit/code-ch04/helper.py) pass: `helper.py:HelperTest:test_int_to_little_endian`

In [128]:
# Exercise 8

reload(helper)
run(helper.HelperTest("test_int_to_little_endian"))

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


### Exercise 9

Create a testnet address for yourself using a long secret that only you know. This is important as there are bots on testnet trying to steal testnet coins. Make sure you write this secret down somewhere! You will be using the secret later to sign Transactions.

In [132]:
# Exercise 9
## NOTE: Fixed a bug with ripemd160. See ecc.py.

reload(ecc)

from ecc import PrivateKey
from helper import hash256, little_endian_to_int

# select a passphrase here, add your email address into the passphrase for security

passphrase = b'your@email.address some secret only you know'
secret = little_endian_to_int(hash256(passphrase))

e = PrivateKey(secret)

print(e.point.address(testnet=True))

# create a private key using your secret
# print an address from the public point of the private key with testnet=True

mtS1mg4oZUJnBWpcSS1vszFoDMjuhbnbnE
